# Step of Implementation
1. Read file csv <br>
2. Convert lat lon to cartesian coordinate(use to cal area with radius in meters) <br>
3. Calculate area of polygon, also area of point <br>
4. Calculate the percentage of intersection <br>
    &emsp;- if percentage of intersection >= 50% then that device in the polygon return IN <br>
    &emsp;- if percentage of intersection  < 50% then the device not in the polygon return OUT <br>
5. Calculate time in the polygon 



## Import libraries
All of libraries are included in this section.

In [1]:
import pandas as pd
import os 
import geopandas as gpd
import numpy as np
import math
from csv import reader
from datetime import datetime
import json

from shapely.geometry import Polygon, LinearRing, Point

In [2]:
#Open the davice data file
device_data = pd.read_csv("dummy.csv")
device_data.head()


,hash_id,latitude,longitude,timestamp,uncertainty
0,1,59.123818,14.608603,1610723283,9.568983
1,1,59.123818,14.608603,1610723284,9.568983
2,1,59.123818,14.608603,1610723285,9.568983
3,1,59.123814,14.608602,1610723286,12.046636
4,1,59.123814,14.608602,1610723287,12.046636


In [3]:
# Converting lat/long to cartesian function

def get_cartesian(lat=None,lon=None):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    R = 6371000 # radius of the earth in meters
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    #z = R *np.sin(lat)
    return x,y


## Convert timestamp to date time and lat,lon
   &emsp;- Convert timestamp to date time <br>
   &emsp;- Convert latitude and longitude to cetesian degree(in meter) and store lat and lon in diffrerent variable
 

In [5]:
convert_timestamp = []
convert_latlon = []
radius = []
convert_latx = []
convert_lony = []
with open('dummy.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    
    if header != None:
    # Iterate over each row in the csv using reader object
        for row in csv_reader:
            #convert timestamp
            timestamp = datetime.fromtimestamp((int (row[3])))
            convert_timestamp.append(timestamp)
            #convert point to cartesian degree
            convert_point = get_cartesian((float(row[1])),(float(row[2])))
            convert_latlon.append(convert_point)
            #store uncertainty as radius
            radius.append(row[4])
            
#store lat, lon in x and y axis
for x in range(len(convert_latlon)):
    convert_latx.append((float(convert_latlon[x][0])))
    convert_lony.append((float(convert_latlon[x][1])))
    
device_data["Datetime"] = convert_timestamp
device_data["Lat_X"] = convert_latx
device_data["Lon_Y"] = convert_lony
device_data

,hash_id,latitude,longitude,timestamp,uncertainty,Datetime,Lat_X,Lon_Y
0,1,59.123818,14.608603,1610723283,9.568983,2021-01-15 16:08:03,3.163800e+06,824615.467933
1,1,59.123818,14.608603,1610723284,9.568983,2021-01-15 16:08:04,3.163800e+06,824615.469664
2,1,59.123818,14.608603,1610723285,9.568983,2021-01-15 16:08:05,3.163800e+06,824615.469664
3,1,59.123814,14.608602,1610723286,12.046636,2021-01-15 16:08:06,3.163800e+06,824615.496292
4,1,59.123814,14.608602,1610723287,12.046636,2021-01-15 16:08:07,3.163800e+06,824615.498387
...,...,...,...,...,...,...,...,...
2217,2,59.123872,14.608547,1610730736,2.000000,2021-01-15 18:12:16,3.163796e+06,824611.057139
2218,2,59.123870,14.608547,1610730765,3.809481,2021-01-15 18:12:45,3.163796e+06,824611.123124
2219,2,59.123889,14.608510,1610730806,3.025978,2021-01-15 18:13:26,3.163795e+06,824608.584434
2220,3,59.123901,14.608552,1610730872,2.000000,2021-01-15 18:14:32,3.163793e+06,824610.647320


## Extract polygon data
The cooridations of the polygon are store in the nested list, so the coordinates need to exrtact and store in tuples. 

In [6]:
# open geojson file
coordinate_polygon = []
with open('outline.geojson') as file:
    data = json.load(file)

#extract nested list of coordinates
for feature in data['features']:
    for list in feature['geometry']['coordinates']:
        for point in list:
        
            point= tuple(point)
            #convert lat lon to catesian
            point = get_cartesian(point[1],point[0])
            coordinate_polygon.append(point)
#convvert to LinearRing for using in Shapely library 
polygon = Polygon(LinearRing(coordinate_polygon))

    

## Calculate circle area for each point
&emsp;- Normally the point has on attribute area. Then, to calculate the intersection area, each point need to add the radius. <br>
&emsp;- The radius in this case is 'uncertainty' column in the dummy.csv file.

In [7]:
#create circle for each point
circle_from_point = []
area_circle = []
intersect_area = []
percentage_of_intersect = []
IN_OUT = []

for i in range(len(convert_latx)):
    #point with radius
    circle = Point((float(convert_latx[i])), (float(convert_lony[i]))).buffer((float(radius[i])))
    area = circle.area
    circle_from_point.append(circle)
    area_circle.append(area)

#calculate the intersection area percentage in each point
for i in range(len(area_circle)):
    point_circle = circle_from_point[i]
    intersect_circle = point_circle.intersection(polygon).area
    percentage_intersect = intersect_circle/point_circle.area*100
    intersect_area.append(intersect_circle)
    percentage_of_intersect.append(percentage_intersect)


#check if the point with radius inside or outside the polygon bt setting the percentage
for i in range(len(percentage_of_intersect)):
    if percentage_of_intersect[i] >= 50:
        IN_OUT.append("IN")
    else:
        IN_OUT.append("OUT")

# add new colum to dataframe
device_data["In_Out"] = IN_OUT
device_data["Percentage_of_Intersection"] = percentage_of_intersect
device_data
device_data.to_csv('device.csv', index=True)

In [8]:
result_df = device_data[(device_data['Percentage_of_Intersection'] >= 70)]
sorted_result = result_df.sort_values(by = ['hash_id', 'Datetime'], ascending = [True, True])


## Select the criteria for the final result
&emsp; - The percentage of area intersection can be classified in this section. <br>
&emsp; - In this case, the percentage of seclection is fixed as equaly and grater than 50%  <br>
&emsp; - The final result is writen in to dataframe first and then can be exported as csv file.<br>

In [11]:
#sorted_result
max_date = []
min_date = []
total_time = []
h_id = []

#select result, show only perccent of intersection more than 50% 
#sorted by hash_id and date time
result_df = device_data[(device_data['Percentage_of_Intersection'] >= 50)]
#sorted_result = result_df.sort_values(by = ['hash_id', 'Datetime'], ascending = [True, True])
grouped = result_df.groupby("hash_id")

result_groups = grouped.groups

#find min max in each hash id
for key, value in grouped: 
    #print(key)
    max_d = max(value['Datetime'])
    min_d = min(value['Datetime'])
    time_diff = max_d - min_d
    max_date.append(max_d)
    min_date.append(min_d)
    total_time.append(time_diff)
    h_id.append(key)
    

In [12]:
# Create new dataframe that contains id of device and the spending time in polygon

final_list = pd.DataFrame(
    {'Hash_ID': h_id,
     'Total_Time': total_time
    })

final_list.to_csv('final_device.csv', index=True)